In [1]:
import pandas as pd
from typing import Literal
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from utils.loader import MovieLensDataset
from arquitecture.Recommender import Recommender
from arquitecture.components.PatterAnalyzer import PatternAnalyzer
import torch.optim as optim

SEED = 55
BATCH = 2
NUN_THREADS = 6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCH = 400
LEARNING_RATE = 0.00001

In [2]:
dataset = MovieLensDataset(ml_path="ml-100k", split="test", seed=SEED)
user_data, train_tensor, test_tensor = dataset.__getitem__(11)
test_tensor

tensor([1.0000, 0.5024, 0.1627, 0.1675, 0.9378, 0.2871, 0.0048, 0.6986, 0.0144,
        0.0813, 0.1675, 0.1340, 0.1722, 0.3780, 0.4785, 0.7225, 0.2297, 0.1053,
        0.0000])

In [3]:
print(f"user_data shape: {user_data.size()}")
print(f"train_tensor shape: {train_tensor.size()}")
print(f"test_tensor shape: {test_tensor.size()}")

user_data shape: torch.Size([23])
train_tensor shape: torch.Size([19, 22])
test_tensor shape: torch.Size([19])


In [4]:
train_dataset = MovieLensDataset(ml_path="ml-100k", split="train", seed=SEED)
test_dataset = MovieLensDataset(ml_path="ml-100k", split="test", seed=SEED)
val_dataset = MovieLensDataset(ml_path="ml-100k", split="val", seed=SEED)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)

for i in range (1):
    for batch in train_dataloader:
        user_data_tensor, rating_train_tensor, rating_test_tensor = batch
        print("  user_data_tensor shape:", user_data_tensor.shape)
        print("  rating_train_tensor shape:", rating_train_tensor.shape)
        print("  rating_test_tensor shape:", rating_test_tensor.shape)
        print(rating_train_tensor)
        break

  user_data_tensor shape: torch.Size([2, 23])
  rating_train_tensor shape: torch.Size([2, 19, 22])
  rating_test_tensor shape: torch.Size([2, 19])
tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 6.4327e-01],
         [7.5000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
          1.0

In [5]:
height=19
width=22
user_input_size = 23
    # Crear instancia del modelo
pattern_analyzer = PatternAnalyzer(conv_structure=[1,4,8,16,32,32,32],
                            input_size=torch.Size((height, width)),
                            pool_depth=3,
                            expert_hidden_size=16,
                            expert_output_len=4,
                            final_mlp_factor=2,
                            final_mlp_output_len=15,
                            ).to(DEVICE)
    
model = Recommender(
        user_data_input_size=user_input_size,
        user_data_analizer_factor = 2,
        user_data_analizer_output_size = 10,
        pattern_analyzer=pattern_analyzer,
        final_regressor_factor=1,
        final_regressor_output_len=19
    ).to(DEVICE)

ratings = torch.randn(628, height, width).to(DEVICE)
user_data = torch.randn(628, user_input_size).to(DEVICE)
    

    # Pasar el tensor por el modelo
output = model(user_data, ratings)
    # Mostrar la forma de la salida final
print("Forma de la salida después de `view`:", output.shape)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

AttributeError: 'ModuleList' object has no attribute 'weight'

In [ ]:
# Crear el optimizador, por ejemplo, usando Adam con una tasa de aprendizaje de 0.001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Crear la función de pérdida para regresión (Mean Squared Error Loss)
criterion = nn.MSELoss(reduction="sum")

In [ ]:
for epoch in range(NUM_EPOCH):
    # --- Training Phase ---
    model.train()  # Set model to training mode
    running_train_loss = 0.0
    
    for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
        optimizer.zero_grad()   
             
        outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE)).to(DEVICE)        
        
        loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE)) 
        
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update model parameters
        running_train_loss += loss.item() * rating_test_tensor.size(0)
    
    epoch_train_loss = running_train_loss / len(train_dataloader.dataset)

    # --- Validation Phase ---
    model.eval()  # Set model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
            
            outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE))        
            loss = criterion(outputs, rating_test_tensor.to(DEVICE)) 
            
            running_val_loss += loss.item() * rating_test_tensor.size(0)
    
    epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCH}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")


patata pre convolucional tensor([[[[5.0000e-01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 7.2447e-03],
          [7.5000e-01, 1.0191e-01, 0.0000e+00, 0.0000e+00, 1.0000e+00,
           0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 1.0000e+00],
          [7.5000e-01, 1.0191e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 8.7971e-01],
          [5.00

KeyboardInterrupt: 